In [240]:
## 2nd MODEL START
import logging

#import matplotlib.pyplot as plt
import tensorflow as tf

from util import encoder, csv_helper, normalizer as norm

N_INPUT = 50000
N_TRAIN = int(N_INPUT / 0.9)
N_TEST = int(N_INPUT / 0.9 * 0.1)
N_CLASSES = 50
LEARNING_RATE = 0.001
N_EPOCHS = 20000
N_STEPS = 100
N_HIDDEN_NEURONS = [10, 12, 24, 48]
N_HIDDEN_LAYERS = len(N_HIDDEN_NEURONS)

In [239]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [241]:
# Anonymous functions for adding sigmoid and softmax layer as wells as
# for initializing variables with zeros and uniform random values between
# -1 and +1.
act = lambda l, w, b: tf.nn.relu(tf.add(tf.matmul(l, w), b))
soft = lambda l, w, b: tf.nn.softmax(tf.add(tf.matmul(l, w), b))
zeros = lambda h: tf.Variable(tf.zeros([h]))
random = lambda i, o: tf.Variable(tf.random_uniform([i, o], -1, 1))

In [242]:
def build_model():
    # Setup neural net: weights, biases and connect layers
    weights = [random(input_size, N_HIDDEN_NEURONS[0])]
    biases = [zeros(N_HIDDEN_NEURONS[0])]
    layers = [act(ph_in, weights[0], biases[0])]
    test_layers = [act(ph_test_in, weights[0], biases[0])]

    # for i in range(1, N_HIDDEN_LAYERS + 1):
    for i in range(1, len(N_HIDDEN_NEURONS)):
        weights.append(random(N_HIDDEN_NEURONS[i - 1], N_HIDDEN_NEURONS[i]))
        biases.append(zeros(N_HIDDEN_NEURONS[i]))
        layers.append(act(layers[i - 1], weights[i], biases[i]))
        test_layers.append(act(test_layers[i - 1], weights[i], biases[i]))

    weights.append(random(N_HIDDEN_NEURONS[N_HIDDEN_LAYERS - 1], N_CLASSES))
    biases.append(zeros(N_CLASSES))
    layers.append(soft(layers[N_HIDDEN_LAYERS - 1], weights[N_HIDDEN_LAYERS], biases[N_HIDDEN_LAYERS]))
    test_layers.append(
        soft(test_layers[N_HIDDEN_LAYERS - 1], weights[N_HIDDEN_LAYERS], biases[N_HIDDEN_LAYERS]))

    return layers[N_HIDDEN_LAYERS], test_layers[N_HIDDEN_LAYERS]

In [243]:
def record():
#     plt.plot(losses, 'b')
#     plt.ylabel('Loss ')
#     plt.draw()
#     plt.pause(0.001)
    logger.info('- Epoch = ' + str(epoch) + ', Loss = ' + str(actual_loss) + ', Train Coverage: ' +
                str(acc_train) + ', Test Coverage: ' + str(acc_test))

In [238]:
if __name__ == "__main__":
    logging.basicConfig(level=logging.INFO, format='%(asctime)s %(levelname)-8s %(message)s')
    logger = logging.getLogger(__name__)
    model_file = "data/trained_models/model-" + str(N_HIDDEN_LAYERS) + "-" + str(N_HIDDEN_NEURONS) + ".ckpt"

    # Read data from dump and map each label to its one hot vector
    csv_input, csv_output = csv_helper._import('data/tagged_questions.csv', N_CLASSES)
    for i in range(0, len(csv_output)):
        csv_output[i] = encoder.one_hot(int(csv_output[i]), N_CLASSES)
    input_size = len(csv_input[0])

    # Split data into training and test set
    train_in, train_out = csv_input[:N_TRAIN], csv_output[:N_TRAIN]
    test_in, test_out = csv_input[N_TRAIN:N_TRAIN + N_TEST], csv_output[N_TRAIN:N_TRAIN + N_TEST]
    train_in, test_in = norm.standard(train_in, test_in)

    # Set TensorFlows placeholder for training input, target value and test input
    ph_in = tf.placeholder(tf.float32, shape=[N_TRAIN, input_size])
    ph_out = tf.placeholder(tf.float32, shape=[N_TRAIN, N_CLASSES])
    ph_test_in = tf.placeholder(tf.float32, shape=[N_TEST, input_size])
    ph_test_out = tf.placeholder(tf.float32, shape=[N_TEST, N_CLASSES])

    net, test_net = build_model()

    # Add node to graph that calculates mean squared error
    LOSS = tf.nn.l2_loss(net - ph_out)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(net, 1), tf.argmax(ph_out, 1)), tf.float32))

    # Initialize optimizer that uses gradient descent
    optimizer = tf.train.AdamOptimizer(LEARNING_RATE).minimize(LOSS)

    saver = tf.train.Saver()
    sess = tf.Session()
    init = tf.global_variables_initializer()
    sess.run(init)

    losses = []
    epoch = 0

    # Run training and check depended on N_STEPS the relative number of correct
    # classified training and test samples as well as plot the loss function in
    # addition to its number of epochs.
    acc_train = sess.run(accuracy, feed_dict={ph_in: train_in, ph_out: train_out})
    while epoch < N_EPOCHS:
        train_dict = {ph_in: train_in, ph_out: train_out}
        test_dict = {ph_test_in: test_in, ph_test_out: test_out}
        sess.run(optimizer, feed_dict=train_dict)
        epoch += 1

        if epoch % N_STEPS == 0:
            acc_train = sess.run(accuracy, feed_dict=train_dict)
            acc_test = sess.run(tf.reduce_mean(
                tf.cast(tf.equal(tf.argmax(test_net, 1), tf.argmax(ph_test_out, 1)), tf.float32)), feed_dict=test_dict)
            actual_loss = sess.run(LOSS, feed_dict=train_dict)
            losses.append(actual_loss)
            record()

    saver.save(sess, model_file)

2020-11-05 19:58:24,656 INFO     - Epoch = 100, Loss = 31552.725, Train Coverage: 0.061650615, Test Coverage: 0.05580558
2020-11-05 19:59:00,401 INFO     - Epoch = 200, Loss = 26431.195, Train Coverage: 0.14709747, Test Coverage: 0.1449145
2020-11-05 19:59:35,847 INFO     - Epoch = 300, Loss = 21958.16, Train Coverage: 0.32123122, Test Coverage: 0.31719172
2020-11-05 20:00:11,441 INFO     - Epoch = 400, Loss = 15619.239, Train Coverage: 0.55386555, Test Coverage: 0.5539154
2020-11-05 20:00:46,831 INFO     - Epoch = 500, Loss = 13742.881, Train Coverage: 0.60595804, Test Coverage: 0.6034203
2020-11-05 20:01:22,024 INFO     - Epoch = 600, Loss = 13011.883, Train Coverage: 0.6322563, Test Coverage: 0.6307831
2020-11-05 20:01:57,191 INFO     - Epoch = 700, Loss = 12560.665, Train Coverage: 0.6494825, Test Coverage: 0.64464444
2020-11-05 20:02:32,442 INFO     - Epoch = 800, Loss = 12233.347, Train Coverage: 0.66260463, Test Coverage: 0.6579658
2020-11-05 20:03:07,472 INFO     - Epoch = 900,

2020-11-05 20:38:18,541 INFO     - Epoch = 7000, Loss = 8981.782, Train Coverage: 0.76417965, Test Coverage: 0.7683168
2020-11-05 20:38:52,660 INFO     - Epoch = 7100, Loss = 8914.21, Train Coverage: 0.76588964, Test Coverage: 0.7710171
2020-11-05 20:39:26,698 INFO     - Epoch = 7200, Loss = 8853.621, Train Coverage: 0.7669157, Test Coverage: 0.77119714
2020-11-05 20:40:00,622 INFO     - Epoch = 7300, Loss = 8795.615, Train Coverage: 0.7686257, Test Coverage: 0.77191716
2020-11-05 20:40:34,391 INFO     - Epoch = 7400, Loss = 8748.1875, Train Coverage: 0.77487177, Test Coverage: 0.7782178
2020-11-05 20:41:08,209 INFO     - Epoch = 7500, Loss = 8706.8955, Train Coverage: 0.7669877, Test Coverage: 0.77191716
2020-11-05 20:41:42,185 INFO     - Epoch = 7600, Loss = 8638.247, Train Coverage: 0.7794618, Test Coverage: 0.7821782
2020-11-05 20:42:16,162 INFO     - Epoch = 7700, Loss = 8580.094, Train Coverage: 0.7797678, Test Coverage: 0.7834383
2020-11-05 20:42:50,152 INFO     - Epoch = 7800, 

2020-11-05 21:17:30,680 INFO     - Epoch = 13900, Loss = 6948.162, Train Coverage: 0.8102601, Test Coverage: 0.81422144
2020-11-05 21:18:05,499 INFO     - Epoch = 14000, Loss = 6943.8735, Train Coverage: 0.8103681, Test Coverage: 0.81476146
2020-11-05 21:18:39,950 INFO     - Epoch = 14100, Loss = 14085.757, Train Coverage: 0.72526324, Test Coverage: 0.7314131
2020-11-05 21:19:14,418 INFO     - Epoch = 14200, Loss = 8822.782, Train Coverage: 0.7892179, Test Coverage: 0.7931593
2020-11-05 21:19:48,856 INFO     - Epoch = 14300, Loss = 7639.6143, Train Coverage: 0.79463595, Test Coverage: 0.79711974
2020-11-05 21:20:23,253 INFO     - Epoch = 14400, Loss = 7622.805, Train Coverage: 0.79517597, Test Coverage: 0.79711974
2020-11-05 21:20:57,651 INFO     - Epoch = 14500, Loss = 7610.748, Train Coverage: 0.795374, Test Coverage: 0.79747975
2020-11-05 21:21:32,062 INFO     - Epoch = 14600, Loss = 7603.9717, Train Coverage: 0.795482, Test Coverage: 0.79747975
2020-11-05 21:22:06,523 INFO     - Ep

KeyboardInterrupt: 

In [ ]:
# Model 2 end

In [228]:
import re
import os
import nltk
import numpy as np
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from tqdm import tqdm
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from gensim.models.keyedvectors import KeyedVectors
word_vect = KeyedVectors.load_word2vec_format("SO_vectors_200.bin", binary=True)

In [224]:
idToTagIndex = {}          #dict mapping post ID to a list of tag indices
tagToTagIndex = {}         #dict mapping tag to tag index

In [225]:
import math
def truncate(number, digits) -> float:
    stepper = 10.0 ** digits
    return math.trunc(stepper * number) / stepper

In [28]:
with open("stacksample/Questions-ascii-1M.csv") as question_input:
    questions_data = pd.read_csv(question_input, engine='python')

    print(questions_data.columns)
    questions_data = questions_data[['Id', 'Body']]
    questions_data.insert(len(questions_data.columns), 'Code', "")
    
    a = re.compile(r'<pre><code>([^<]*)</code></pre>')
    b = re.compile(r'<.*?>')
    stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])
    
    questions_data['Code'] = questions_data['Body'].apply(lambda x: ' '.join(re.findall(a, x)))

    def clean(text):
        # remove the code from the text
        x = re.sub(a, '', text)
        
        # removes the parantheses from the text
        x = re.sub(b, '', x)
        
        # replace multiple new lines
        x = x.replace('\n\n', '\n')
        
        # specific word converter
        x = re.sub(r"won't", "will not", x)
        x = re.sub(r"can\'t", "can not", x)

        # generalized words converter
        x = re.sub(r"n\'t", " not", x)
        x = re.sub(r"\'re", " are", x)
        x = re.sub(r"\'s", " is", x)
        x = re.sub(r"\'d", " would", x)
        x = re.sub(r"\'ll", " will", x)
        x = re.sub(r"\'t", " not", x)
        x = re.sub(r"\'ve", " have", x)
        x = re.sub(r"\'m", " am", x)
        
        #remove stopwords
        x = ' '.join(e.lower() for e in x.split() if e.lower() not in stopwords)
        
        return x
    questions_data['Body'] = questions_data['Body'].apply(clean)
    
#     questions_data['Body'] = questions_data['Body'].apply(nltk.tokenize.word_tokenize) #need to fix: don't convert
#     # C#, C++ to C
#     questions_data['Body'] = questions_data['Body'].apply(lambda x: [word for word in x if word.isalnum()])
#     questions_data['Body'] = questions_data['Body'].apply(lambda x: [word.lower() for word in x])

Index(['Id', 'OwnerUserId', 'CreationDate', 'ClosedDate', 'Score', 'Title',
       'Body'],
      dtype='object')


In [29]:
from collections import defaultdict

idToTagIndex = {}          #dict mapping post ID to a list of tag indices
tagToTagIndex = {}         #dict mapping tag to tag index
tagIndexToTag = {}
tagToFrequency = defaultdict(lambda: 0)

with open("stacksample/tags-1M.csv") as tag_input:
    tag_data = pd.read_csv(tag_input)
    tagIndex = 0
    for index, row in tqdm(tag_data.iterrows()):
        currId = int(row[0])
        currTag = row[1]
        if currTag not in tagToTagIndex:
            tagToTagIndex[currTag] = tagIndex
            tagIndexToTag[tagIndex] = currTag
            currTagIndex = tagIndex
            tagIndex += 1

        else:
            currTagIndex = tagToTagIndex[currTag]  
        
        tagToFrequency[currTagIndex] += 1        
    
        if currId not in idToTagIndex.keys():
            idToTagIndex[currId] = [tagToTagIndex[row[1]]]
        else:
            idToTagIndex[currId].append(tagToTagIndex[row[1]])
            

152759it [00:13, 11181.25it/s]


In [15]:
print("Number of examples: ", len(idToTagIndex))

Number of examples:  53203


In [30]:
# find 10 most common tags

tagToFrequencyList = []

for key, value in tagToFrequency.items():
    temp = [key, value]
    tagToFrequencyList.append(temp)
    
tagToFrequencyList.sort(reverse=True, key=lambda x: x[1])

for tag in tagToFrequencyList[:10]:
    print(f"{tagIndexToTag[tag[0]]} ({tag[0]}): {tag[1]} times")
    
mostCommonTags = {}
for counter, tag in enumerate(tagToFrequencyList[:10]):   #currently takes top 10 tags
    mostCommonTags[tag[0]] = counter


c# (14): 6722 times
java (89): 3858 times
.net (15): 3598 times
php (76): 3223 times
asp.net (8): 3041 times
javascript (132): 2852 times
c++ (18): 2509 times
jquery (370): 2198 times
iphone (607): 2111 times
python (196): 2070 times


In [31]:
# list(idToTagIndex.values())[:10]
print(mostCommonTags)

{14: 0, 89: 1, 15: 2, 76: 3, 8: 4, 132: 5, 18: 6, 370: 7, 607: 8, 196: 9}


In [32]:
idToTenTags = {}

for postId, tags in idToTagIndex.items():
    containsTopTenTags = [mostCommonTags[tag] for tag in tags if tag in mostCommonTags.keys()]
    idToTenTags[postId] = containsTopTenTags
    
questions_data['Top-Tags'] = questions_data['Id'].apply(lambda x: idToTenTags[x])

print(questions_data)

            Id                                               Body  \
0           80  I've written a database generation script in S...   
1           90  Are there any really good tutorials explaining...   
2          120  Has anyone got experience creating SQL-based A...   
3          180  This is something I've pseudo-solved many time...   
4          260  I have a little game written in C#. It uses a ...   
...        ...                                                ...   
53198  2495810  I'm going to host an app on a shared host and ...   
53199  2495870  I use MouseMove event to move objects(say labe...   
53200  2495890  I have dragged a empty asp.net table onto my w...   
53201  2495910  I want to log some seemingly random errors I'm...   
53202  2496040  I have a php array that has a bunch of data th...   

                                                    Code Top-Tags  
0      Create Table tRole (\n      roleID integer Pri...       []  
1                                  

In [57]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

vectorizer = TextVectorization(max_tokens=20000, output_sequence_length=200)
text_ds = tf.data.Dataset.from_tensor_slices(questions_data['Body'].values.tolist())
vectorizer.adapt(text_ds)
vectorizer.get_vocabulary()[:10]

['', '[UNK]', 'the', 'to', 'i', 'a', 'is', 'and', 'in', 'of']

In [170]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()

questions_X = vectorizer(np.array([[s] for s in questions_data['Body'].values])).numpy()
questions_y = mlb.fit_transform(np.array(questions_data['Top-Tags'].values))
print(questions_y[:5])

train_X, test_X, train_y, test_y = train_test_split(questions_X, questions_y, train_size=0.90, random_state=200)

[[0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [1 0 1 0 0 0 0 0 0 0]]
47882


In [159]:
output = vectorizer([["I tried running this line of code, but I'm receiving a null pointer exception"]])
output.numpy()

array([[   4,  124,  176,   12,  166,    9,   40,   15,   26, 1701,    5,
         393,  848,  291,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0, 

In [160]:
voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))
print(word_index["hello"])

1517


In [161]:
test = ["this", "line", "of", "code"]
[word_index[w] for w in test] 

[12, 166, 9, 40]

In [162]:
num_tokens = len(voc) + 2
print(num_tokens)
embedding_dim = 200
hits = 0
misses = 0

embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    try:
        embedding_vector = word_vect.get_vector(word)
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    except:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

20002
Converted 18657 words (1343 misses)


In [163]:
from tensorflow.keras.layers import Embedding

embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
)

In [217]:
from tensorflow.keras import layers, Sequential

int_sequences_input = keras.Input(shape=(None,), dtype="int64")
embedded_sequences = embedding_layer(int_sequences_input)
model = Sequential()
model.add(Embedding(num_tokens, embedding_dim))
model.add(layers.LSTM(128, return_sequences=True))
model.add(layers.Dropout(0.5))
model.add(layers.LSTM(64))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(len(mostCommonTags), activation='sigmoid'))

# model = Sequential()
# model.add(embedding_layer)
# model.add(layers.SpatialDropout1D(0.2))
# model.add(layers.LSTM(100, dropout=0.2, recurrent_dropout=0.2))
# model.add(layers.Dense(len(mostCommonTags), activation='sigmoid'))

model.summary()

Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 200)         4000400   
_________________________________________________________________
spatial_dropout1d_13 (Spatia (None, None, 200)         0         
_________________________________________________________________
lstm_14 (LSTM)               (None, 100)               120400    
_________________________________________________________________
dense_69 (Dense)             (None, 10)                1010      
Total params: 4,121,810
Trainable params: 121,410
Non-trainable params: 4,000,400
_________________________________________________________________


In [222]:
from sklearn.utils import class_weight
import tensorflow.keras.backend as K

import tensorflow.keras.callbacks as cbks
class CustomMetrics(cbks.Callback):

    def on_epoch_end(self, epoch, logs=None):
        for k in logs:
            print(logs[k])

def custom_metric(y_true, y_pred):
    print("y_true")
    print(K.print_tensor(y_true))
    print("y_pred")
    print(K.print_tensor(y_pred))
    
  
    

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              #loss = custom_loss,
              metrics = [tf.keras.metrics.TopKCategoricalAccuracy(k=3)]) 
              #metrics = 'accuracy')
              #metrics = custom_metric)                

weights = {}

for i in range(10):
    weights[i] = (1 / tagToFrequencyList[i][1])

model.fit(train_X, train_y, batch_size=128, epochs=1, validation_data=(test_X, test_y), class_weight=weights, callbacks = [])

375/375 [==============================] - 218s 580ms/step - loss: 6.2214e-05 - top_k_categorical_accuracy: 0.4550 - val_loss: 0.2313 - val_top_k_categorical_accuracy: 0.1853


In [221]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

# string_input = keras.Input(shape=(200,), dtype="string")
predictions=model.predict([test_X])
# print("test_x")
# print(test_X)
#thresholds=[0.01,0.02, 0.03, 0.04, 0.05, 0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
thresholds = [0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.1]
for val in thresholds:
    pred=predictions.copy()
#     print("pred before")
#     print(pred)
    pred[pred>=val]=1
    pred[pred<val]=0  
#     print("val: {}", val)
#     print(test_y.shape)
#     print(pred.shape)
#     print(test_X.shape)
#     print("----")
#     print("test_y")
#     print(test_y)
#     print("pred after")
#     print(pred)
    precision = precision_score(test_y, pred, average='micro')
    recall = recall_score(test_y, pred, average='micro')
    f1 = f1_score(test_y, pred, average='micro')
   
    print("Micro-average quality numbers for threshold: " + str(val))
    print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))


Micro-average quality numbers for threshold: 0.01
Precision: 0.0609, Recall: 1.0000, F1-measure: 0.1149
Micro-average quality numbers for threshold: 0.02
Precision: 0.0609, Recall: 1.0000, F1-measure: 0.1149
Micro-average quality numbers for threshold: 0.03
Precision: 0.0610, Recall: 0.9997, F1-measure: 0.1149
Micro-average quality numbers for threshold: 0.04
Precision: 0.0610, Recall: 0.9969, F1-measure: 0.1149
Micro-average quality numbers for threshold: 0.05
Precision: 0.0610, Recall: 0.9920, F1-measure: 0.1149
Micro-average quality numbers for threshold: 0.06
Precision: 0.0612, Recall: 0.9880, F1-measure: 0.1152
Micro-average quality numbers for threshold: 0.07
Precision: 0.0617, Recall: 0.7936, F1-measure: 0.1145
Micro-average quality numbers for threshold: 0.08
Precision: 0.0648, Recall: 0.4198, F1-measure: 0.1123
Micro-average quality numbers for threshold: 0.09
Precision: 0.0713, Recall: 0.0123, F1-measure: 0.0210
Micro-average quality numbers for threshold: 0.1
Precision: 0.05

In [220]:
string_input = keras.Input(shape=(1,), dtype="string")
x = vectorizer(string_input)
preds = model(x)
end_to_end_model = keras.Model(string_input, preds)

probabilities = end_to_end_model.predict(
#    ["I wanted to open my app without safari system alert but I found out that is impossible. so i decided to handle this alert event but I couldn't find the way. if I click [open], then safari open App, but if I click [cancel], then 'appCheckTimer' will be executed, then safari moves to 'some page's url'. if there is no way to not open this alert, I want to handle this alert's button event, when user click [cancel], I just want to stay that page. that alert is not opened by me, it's by safari So I can't handle it."]
#    ["Dropped my iphone again :("]
    ["my name is Varuni"]
)

for i, prob in np.ndenumerate(probabilities):
    print('{:<16}  {:<16}'.format(tagIndexToTag[tagToFrequencyList[i[1]][0]], truncate(prob, 3)))
prob = tagIndexToTag[tagToFrequencyList[np.argmax(probabilities)][0]]
val = prob if np.max(probabilities) >= 0.5 else None
print(f"\nMost likely tag: {val}")

c#                0.083           
java              0.072           
.net              0.062           
php               0.071           
asp.net           0.066           
javascript        0.084           
c++               0.074           
jquery            0.082           
iphone            0.072           
python            0.084           

Most likely tag: None
